In [1]:
import pubchempy as pcp
import pandas as pd
import argparse
import os
import time

In [ ]:
def get_compound_from_smiles(smiles):
    attempts = 5
    time_delay = 1 # in seconds
    while attempts >= 1:
        try:
            compounds = pcp.get_compounds(smiles, namespace='smiles')
            cid = compounds[0].cid
            if cid == None:
                print('No PubChem record') # https://pubchempy.readthedocs.io/en/latest/guide/gettingstarted.html
                return None
            compound = pcp.Compound.from_cid(cid)
        except:
            attempts -= 1
            print('Could not get compound. ' + str(attempts) + ' attempts remaining.')
            time.sleep(time_delay)
        else:
            return compound
    
    if attempts <= 0:
        print('Failed to get compound from smiles after exhausting all attempts')
        return None

def compound_series(smiles_file):
    compound_series_list = []
    for index, line in enumerate(smiles_file):
        # parse line
        parsed_line = line.strip().split('\t')
        smiles, ncats_id, label = tuple(parsed_line)
        
        # get compound
        print('Getting compound ' + str(index))
        compound = get_compound_from_smiles(smiles)
        
        # check for empty compound (e.g. failed to get compound from smiles code)
        if compound == None:
            continue
        
        # convert compound to pandas Series
        compound_series = compound.to_series()
        
        # append label, smiles, and ncats_id to pandas Series
        compound_series['label'] = label
        compound_series['smiles'] = smiles
        compound_series['ncats_id'] = ncats_id
        
        # add Series to list of compound Series
        compound_series_list.append(compound_series)
    
    return compound_series_list

def compound_series_to_dataframe(compound_series_list):
    df = pd.concat(compound_series_list, axis=1)
    df = df.T
    # df = df.set_index('cid')
    return df

In [ ]:
input_filename = 'train/sr-are'
input_directory = 'data/'
output_directory = 'features-pandas/'
output_filename = output_directory + input_filename + '.features'

# get list of pandas Series
with open('data/'+ input_filename + ".smiles", 'r') as smiles_file:
    compound_series_list = compound_series(smiles_file)

df = compound_series_to_dataframe(compound_series_list)
df.to_csv(output_filename, sep='\t')

Getting compound 0
Getting compound 1
Getting compound 2
Getting compound 3
Getting compound 4
Getting compound 5
Getting compound 6
Getting compound 7
Getting compound 8
Getting compound 9
Getting compound 10
Getting compound 11
Getting compound 12
Getting compound 13
Getting compound 14
Getting compound 15
Getting compound 16
Getting compound 17
Getting compound 18
Getting compound 19
Getting compound 20
Getting compound 21
Getting compound 22
Getting compound 23
Getting compound 24
Getting compound 25
Getting compound 26
Getting compound 27
Getting compound 28
Getting compound 29
Getting compound 30
Getting compound 31
Getting compound 32
Getting compound 33
Getting compound 34
Getting compound 35
Getting compound 36
Getting compound 37
Getting compound 38
Getting compound 39
Getting compound 40
Getting compound 41
Getting compound 42
Getting compound 43
Getting compound 44
Getting compound 45
Getting compound 46
Getting compound 47
Getting compound 48
Getting compound 49
Getting co

Getting compound 395
Getting compound 396
Getting compound 397
Getting compound 398
Getting compound 399
Getting compound 400
Getting compound 401
Getting compound 402
Getting compound 403
Getting compound 404
Getting compound 405
Getting compound 406
Getting compound 407
Getting compound 408
Getting compound 409
Getting compound 410
Getting compound 411
Getting compound 412
Getting compound 413
Getting compound 414
Getting compound 415
Getting compound 416
Getting compound 417
Getting compound 418
Getting compound 419
Getting compound 420
Getting compound 421
Getting compound 422
Getting compound 423
Getting compound 424
Getting compound 425
Getting compound 426
Getting compound 427
Getting compound 428
Getting compound 429
Getting compound 430
Getting compound 431
Getting compound 432
Getting compound 433
Getting compound 434
Getting compound 435
Getting compound 436
Getting compound 437
Getting compound 438
Getting compound 439
Getting compound 440
Getting compound 441
Getting compo

Getting compound 786
Getting compound 787
Getting compound 788
Getting compound 789
Getting compound 790
Getting compound 791
Getting compound 792
Getting compound 793
Getting compound 794
Getting compound 795
Getting compound 796
Getting compound 797
Getting compound 798
Getting compound 799
Getting compound 800
Getting compound 801
Getting compound 802
Getting compound 803
Getting compound 804
Getting compound 805
Getting compound 806
Getting compound 807
Getting compound 808
Getting compound 809
Getting compound 810
Getting compound 811
Getting compound 812
Getting compound 813
Getting compound 814
Getting compound 815
Getting compound 816
Getting compound 817
Could not get compound. 4 attempts remaining.
Could not get compound. 3 attempts remaining.
Could not get compound. 2 attempts remaining.
Could not get compound. 1 attempts remaining.
Could not get compound. 0 attempts remaining.
Failed to get compound from smiles after exhausting all attempts
Getting compound 818
Could not ge

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.


KeyboardInterrupt

ERROR:tornado.general:Uncaught exception, closing connection.
Traceback (most recent call last):
  File "C:\Miniconda3_64\envs\cs221_project_py36x64\lib\site-packages\zmq\eventloop\zmqstream.py", line 414, in _run_callback
    callback(*args, **kwargs)
  File "C:\Miniconda3_64\envs\cs221_project_py36x64\lib\site-packages\tornado\stack_context.py", line 277, in null_wrapper
    return fn(*args, **kwargs)
  File "C:\Miniconda3_64\envs\cs221_project_py36x64\lib\site-packages\ipykernel\kernelbase.py", line 283, in dispatcher
    return self.dispatch_shell(stream, msg)
  File "C:\Miniconda3_64\envs\cs221_project_py36x64\lib\site-packages\ipykernel\kernelbase.py", line 235, in dispatch_shell
    handler(stream, idents, msg)
  File "C:\Miniconda3_64\envs\cs221_project_py36x64\lib\site-packages\ipykernel\kernelbase.py", line 421, in execute_request
    self._abort_queue